In [1]:
from pyopenms import *
import os
import glob

Determination of memory status is not supported on this 
 platform, measuring for memoryleaks will never fail


In [2]:
path= "results/interim"
if not os.path.exists(path):
    os.mkdir(path)

In [3]:
# 1) Precursor Mass Correction

input_original_files = glob.glob('Example_data/*.mzML')

for filename in input_original_files:
    print('Processing: ', filename)
    exp = MSExperiment()
    MzMLFile().load(filename, exp)
    exp.sortSpectra(True)
    delta_mzs= []
    mzs = []
    rts= []
    PrecursorCorrection.correctToHighestIntensityMS1Peak(exp, 100.0, True, delta_mzs, mzs, rts)
    MzMLFile().store(os.path.join(path, 'PCpeak_' + os.path.basename(filename)), exp)
print('Finished Precursor Mass Correction')

Processing:  Example_data/Actino_1.mzML
Processing:  Example_data/Actino_2.mzML
Finished Precursor Mass Correction


In [4]:
# 2) Feature Detection

input_mzml_files = glob.glob('results/interim/PCpeak_*.mzML')

# 2.1) Mass trace detection

for filename in input_mzml_files:
    print("Mass Trace Detection: ", filename)
    exp = MSExperiment()
    MzMLFile().load(filename, exp)
    exp.sortSpectra(True)
    mass_traces = []
    mtd = MassTraceDetection()
    mtd_par = mtd.getDefaults()
    mtd_par.setValue("mass_error_ppm", 10.0) 
    mtd_par.setValue("noise_threshold_int", 1.0e04)
    mtd.setParameters(mtd_par)
    mtd.run(exp, mass_traces, 0)

# 2) Elution peak detection
    print("Elution Peak Detection: ", filename)
    mass_traces_split = []
    mass_traces_final = []
    epd = ElutionPeakDetection()
    epd_par = epd.getDefaults()
    epd_par.setValue("width_filtering", "fixed")
    epd.setParameters(epd_par)
    epd.detectPeaks(mass_traces, mass_traces_split)
     
    if (epd.getParameters().getValue("width_filtering") == "auto"):
          epd.filterByPeakWidth(mass_traces_split, mass_traces_final)
    else:
          mass_traces_final = mass_traces_split

# 3) Feature detection
    print("Feature Detection: ", filename)
    feature_map_FFM = FeatureMap()
    feat_chrom = []
    ffm = FeatureFindingMetabo()
    ffm_par = ffm.getDefaults() 
    ffm_par.setValue("isotope_filtering_model", "none")
    ffm_par.setValue("remove_single_traces", "true")
    ffm_par.setValue("mz_scoring_by_elements", "false")
    ffm_par.setValue("report_convex_hulls", "true")
    ffm.setParameters(ffm_par)
    ffm.run(mass_traces_final, feature_map_FFM, feat_chrom)
    feature_map_FFM.setUniqueIds()
    feature_map_FFM.setPrimaryMSRunPath([filename.encode()])
    print(filename[7:-5] + ".featureXML")
    FeatureXMLFile().store(os.path.join(path, os.path.basename(filename)[7:-5] + ".featureXML"), feature_map_FFM)
    
print("Finished Feature Detection")

Mass Trace Detection:  results/interim/PCpeak_Actino_2.mzML
Progress of 'Elution Peak Detection: mass trace detection':
-- done [took 0.27 s (CPU), 0.10 s (Wall)] -- 
 results/interim/PCpeak_Actino_2.mzML
Progress of 'elution peak detection':Feature Detection: 
-- done [took 0.61 s (CPU), 0.18 s (Wall)] -- 
 results/interim/PCpeak_Actino_2.mzML
Progress of 'assembling mass traces to features':/interim/PCpeak_Actino_2.featureXML

-- done [took 0.48 s (CPU), 0.27 s (Wall)] -- 
Mass Trace Detection:  results/interim/PCpeak_Actino_1.mzML
Progress of 'mass trace detectionElution Peak Detection: ':
-- done [took 0.75 s (CPU), 0.12 s (Wall)] -- 
 results/interim/PCpeak_Actino_1.mzML
Progress of 'elution peak detection':
-- done [took 1.18 s (CPU), 0.19 s (Wall)] -- 
Feature Detection:  results/interim/PCpeak_Actino_1.mzML
Progress of 'assembling mass traces to features':
-- done [took 0.89 s (CPU), 0.14 s (Wall)] -- 
/interim/PCpeak_Actino_1.featureXML
Finished Feature Detection


In [5]:
input_features = sorted(glob.glob('results/interim/*.featureXML'))

for consensus in input_features:
    fmap = FeatureMap()
    FeatureXMLFile().load(consensus, fmap)
    DF= fmap.get_df()
    DF= DF.drop(columns=["ID_native_id", "peptide_score", "peptide_sequence", "ID_filename"])
    DF=DF.rename(columns={"f1":"intensity"})
print("example:", os.path.basename(consensus))
display(DF)

AttributeError: 'pyopenms.pyopenms_4.FeatureMap' object has no attribute 'get_df'